In [1]:
import pandas as pd
import numpy as np
import warnings
import time
import csv
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import csv
from textblob import TextBlob
import pandas
import sklearn
#import cPickle
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
#from sklearn.grid_search import GridSearchCV
#from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 
from sklearn.tree import DecisionTreeClassifier 
#from sklearn.learning_curve import learning_curve

In [3]:
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import auc, plot_precision_recall_curve

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

**Loading the train, test and validation data from the saved** **CSVs**

In [5]:
path = "C:\\Users\\Neha\\Documents\\4th Sem\\AML\\Spam_detection\\data\\"
df_train = pd.read_csv(path + "train.csv")
df_test = pd.read_csv(path + "test.csv")
df_valid = pd.read_csv(path + "valid.csv")

In [6]:
x_train = df_train["x_train"]
y_train = df_train["y_train"]

In [7]:
x_test = df_test["x_test"]
y_test = df_test["y_test"]

In [8]:
x_valid = df_valid["x_valid"]
y_valid = df_valid["y_valid"]

**Function to split the messages into words**

In [9]:
def split_into_lemmas(message):
    message = str(message).lower()
    words = TextBlob(message).words
    return [word.lemma for word in words]

In [10]:
import nltk

In [11]:
import mlflow

**Decision Tree Classifier**

---



In [12]:
EXPERIMENT_NAME = "mlflow-spam-detection_011"
EXPERIMENT_ID = mlflow.create_experiment(EXPERIMENT_NAME)

In [13]:
for idx, depth in enumerate([1, 2, 5, 10, 20]):
    #clf = DecisionTreeClassifier(max_depth=depth)
    pipeline_decisicionTree = Pipeline([
    ('bow', CountVectorizer(analyzer=split_into_lemmas)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', DecisionTreeClassifier(random_state=42,max_depth = depth)), 
      ])
    #clf.fit(x_train, y_train)
    pipeline_decisicionTree.fit(x_train,y_train)
    #y_pred = clf.predict(x_test)
    y_pred_tree = pipeline_decisicionTree.predict(x_valid)
    report = classification_report(y_valid, y_pred_tree)
    accuracy = accuracy_score(y_valid, y_pred_tree)
    precision, recall, thresholds = precision_recall_curve(y_valid, y_pred_tree)
    auc_precision_recall = auc(recall, precision)

    
    # Start MLflow
    RUN_NAME = f"run_{idx}"
    with mlflow.start_run(experiment_id=EXPERIMENT_ID, run_name=RUN_NAME) as run:
        # Retrieve run id
        RUN_ID = run.info.run_id

        # Track parameters
        mlflow.log_param("depth", depth)

        # Track metrics
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("AUCPR", auc_precision_recall)

        # Track model
        mlflow.sklearn.log_model(pipeline_decisicionTree, "classifier")

C:\Users\Neha\anaconda3\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [14]:
from mlflow.tracking import MlflowClient

In [15]:
def print_run_info(runs):
    for r in runs:
        print("run_id: {}".format(r.info.run_id))
        print("lifecycle_stage: {}".format(r.data.params))
        print("metrics: {}".format(r.data.metrics))

        # Exclude mlflow system tags
        tags = {k: v for k, v in r.data.tags.items() if not k.startswith("mlflow.")}
        print("tags: {}".format(tags))


In [16]:
client = MlflowClient()

In [17]:
runs = client.search_runs(EXPERIMENT_ID, order_by=["metrics.m DESC"])
print_run_info(runs)
print("--")

run_id: d1284316d39f4451bc5097c9af9c0393
lifecycle_stage: {'depth': '20'}
metrics: {'accuracy': 0.9658886894075404, 'AUCPR': 0.8732346475348474}
tags: {}
run_id: bc26cc823bd745878ae1deea56911792
lifecycle_stage: {'depth': '10'}
metrics: {'accuracy': 0.9712746858168761, 'AUCPR': 0.893947999556516}
tags: {}
run_id: 719981ba3b5347319b84e40cc7d94618
lifecycle_stage: {'depth': '5'}
metrics: {'accuracy': 0.9533213644524237, 'AUCPR': 0.8255091180344677}
tags: {}
run_id: 2e11593c2c9a47c9891265ec7670a077
lifecycle_stage: {'depth': '2'}
metrics: {'accuracy': 0.926391382405745, 'AUCPR': 0.7134064835991745}
tags: {}
run_id: 4a4b4e86f51d467f8bdd99dc900384ef
lifecycle_stage: {'depth': '1'}
metrics: {'accuracy': 0.8707360861759426, 'AUCPR': 0.4939735537751926}
tags: {}
--


**Naive Bayes Model**

---




In [18]:
EXPERIMENT_NAME_BM = "mlflow-spam-detection-bayes_0110"
EXPERIMENT_ID_BM = mlflow.create_experiment(EXPERIMENT_NAME_BM)

In [19]:
for idx, depth in enumerate([0.01,0.05,0.1,0.5,1]):
    #clf = DecisionTreeClassifier(max_depth=depth)
    pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=split_into_lemmas)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB(alpha = depth)),  # train on TF-IDF vectors w/ Naive Bayes classifier
        ])
    #clf.fit(x_train, y_train)
    pipeline.fit(x_train,y_train)
    #y_pred = clf.predict(x_test)
    y_pred_NB = pipeline.predict(x_valid)
    report = classification_report(y_valid, y_pred_NB)
    accuracy = accuracy_score(y_valid, y_pred_NB)
    precision, recall, thresholds = precision_recall_curve(y_valid, y_pred_NB)
    auc_precision_recall = auc(recall, precision)

    # Start MLflow
    RUN_NAME_1 = f"run_{idx}"
    with mlflow.start_run(experiment_id=EXPERIMENT_ID_BM, run_name=RUN_NAME_1) as run:
        # Retrieve run id
        RUN_ID = run.info.run_id

        # Track parameters
        mlflow.log_param("depth", depth)

        # Track metrics
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("AUCPR", auc_precision_recall)

        # Track model
        mlflow.sklearn.log_model(pipeline, "classifier")

In [20]:
client2 = MlflowClient()

In [21]:
runs_BM = client2.search_runs(EXPERIMENT_ID_BM, order_by=["metrics.m DESC"])
print_run_info(runs_BM)
print("--")

run_id: 1d6b40300d4d48bfa6898582c849fca2
lifecycle_stage: {'depth': '1'}
metrics: {'accuracy': 0.9551166965888689, 'AUCPR': 0.8463853136773966}
tags: {}
run_id: 737e3fc9174546d0a26106c9d00a8b73
lifecycle_stage: {'depth': '0.5'}
metrics: {'accuracy': 0.9748653500897666, 'AUCPR': 0.9139757756593421}
tags: {}
run_id: 997cd45125bb450e95060fe99a14a851
lifecycle_stage: {'depth': '0.1'}
metrics: {'accuracy': 0.9910233393177738, 'AUCPR': 0.9692770627354793}
tags: {}
run_id: f13b49891d8e4e33bd1557c3b79c71dd
lifecycle_stage: {'depth': '0.05'}
metrics: {'accuracy': 0.9910233393177738, 'AUCPR': 0.9692770627354793}
tags: {}
run_id: 75d6a4c9179541559a1e9c463ac46165
lifecycle_stage: {'depth': '0.01'}
metrics: {'accuracy': 0.9910233393177738, 'AUCPR': 0.9692770627354793}
tags: {}
--


**Random Forest Classifier**

---



In [22]:
EXPERIMENT_NAME_RF = "mlflow-spam-detection-random-forestnew_0011"
EXPERIMENT_ID_RF = mlflow.create_experiment(EXPERIMENT_NAME_RF)

In [23]:
from sklearn.ensemble import RandomForestClassifier

In [24]:
for idx, depth in enumerate([1, 2, 5, 10, 20]):
    #clf = DecisionTreeClassifier(max_depth=depth)
    
    
    pipeline_random_forest = Pipeline([
    ('bow', CountVectorizer(analyzer=split_into_lemmas)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', RandomForestClassifier(random_state=42,max_depth = depth)),  # train on TF-IDF vectors w/ Naive Bayes classifier
      ])
    
    pipeline_random_forest.fit(x_train,y_train)
    #y_pred = clf.predict(x_test)
    y_pred_random_forest = pipeline_random_forest.predict(x_valid)
    report = classification_report(y_valid, y_pred_random_forest)
    accuracy_random_forest = accuracy_score(y_valid, y_pred_random_forest)
    precision, recall, thresholds = precision_recall_curve(y_valid, y_pred_tree)
    auc_precision_recall = auc(recall, precision)

    # Start MLflow
    RUN_NAME_RF = f"run_{idx}"
    with mlflow.start_run(experiment_id=EXPERIMENT_ID_RF, run_name=RUN_NAME_RF) as run:
        # Retrieve run id
        RUN_ID = run.info.run_id

        # Track parameters
        mlflow.log_param("depth", depth)

        # Track metrics
        mlflow.log_metric("accuracy_rf", accuracy_random_forest)
        mlflow.log_metric("AUCPR", auc_precision_recall)

        # Track model
        mlflow.sklearn.log_model(pipeline_random_forest, "classifier")

C:\Users\Neha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Neha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Neha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Neha\anaconda3\lib\site-packages\sklearn\metrics\

In [46]:
client3 = MlflowClient()

In [47]:
runs_1 = client3.search_runs(EXPERIMENT_ID_RF, order_by=["metrics.m DESC"])
print_run_info(runs_1)
print("--")

run_id: 1bd945df0b554eeb8927cc68e15c7e9b
lifecycle_stage: {'depth': '20'}
metrics: {'accuracy_rf': 0.9569120287253142}
tags: {}
run_id: a9811a7b86be447d98bb8d8d72e59d8e
lifecycle_stage: {'depth': '10'}
metrics: {'accuracy_rf': 0.9102333931777379}
tags: {}
run_id: 95ae06887db3440cb687b5aca92ec432
lifecycle_stage: {'depth': '5'}
metrics: {'accuracy_rf': 0.874326750448833}
tags: {}
run_id: f68a6054925b4a9b90933f4274a37a27
lifecycle_stage: {'depth': '2'}
metrics: {'accuracy_rf': 0.8725314183123878}
tags: {}
run_id: eef299c1acbc45a5a6c4459b3caeb352
lifecycle_stage: {'depth': '1'}
metrics: {'accuracy_rf': 0.8725314183123878}
tags: {}
--
